In [18]:
%reload_ext autoreload
%autoreload 2

from glob import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences
from scipy.signal import chirp, find_peaks, peak_widths, peak_prominences
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
from ECG_features_time import * 
from EDA_Features2 import *
from TEMP import *
from ECG_features_freq import *
from EMG_Features import *
from Remove_Movement import *
from Resp_Features import *
from wesad import read_data_of_one_subject
import warnings
import numpy as np
import pandas as pd
from RESP import RESPprep



C4_subjects = ["P1", "P2", "P5", "P9", "P10", "P11", "P12"]
C4_gender = ['2', '-2', '2', '2', '-2', '-2', '2']




for frame in np.arange(30,125,5):
    all_data_df = pd.DataFrame()

    for item in C4_subjects:

        index_in_list = C4_subjects.index(item)
        EXT = item + "_*"
        all_csv_files = [file for path, subdir, files in os.walk("C4-Processed") for file in glob(os.path.join(path, EXT))]
        all_csv_files.sort()

        print(item)
        df_base = pd.read_csv(all_csv_files[0], header = ([0]))
        df_stress = pd.read_csv(all_csv_files[1], header = ([0]))


        patient_df = pd.DataFrame()
        base_dict = {}
        stress_dict = {}
        obj_data = {}

        ecg_base = df_base['ECG'].to_numpy()
        ecg_base = ecg_base - np.mean(ecg_base)
        emg_base = df_base['EMG'].to_numpy()
        eda_base = df_base['EDA'].to_numpy()
        temp_base = df_base['Temp'].to_numpy()
        acc_base = df_base[['ACC_X', 'ACC_Y', 'ACC_Z']].to_numpy()
        resp_base = df_base['RESP'].dropna().to_numpy()

        ecg_stress = df_stress['ECG'].to_numpy()
        ecg_stress = ecg_stress - np.mean(ecg_stress)
        emg_stress = df_stress['EMG'].to_numpy()
        eda_stress = df_stress['EDA'].to_numpy()
        temp_stress = df_stress['Temp'].to_numpy()
        acc_stress = df_stress[['ACC_X', 'ACC_Y', 'ACC_Z']].to_numpy()
        resp_stress = df_stress['RESP'].dropna().to_numpy()


        ecg_features_time_base = ECG_time_data(ecg_base, frame)
        ecg_features_time_stress = ECG_time_data(ecg_stress, frame)


        eda_features_base = calc_eda_features(eda_base, frame)
        eda_features_stress = calc_eda_features(eda_stress, frame)

        emg_features_base = calc_emg_features(emg_base, frame)
        emg_features_stress = calc_emg_features(emg_stress, frame)

        temp_features_base = calc_temp_features(temp_base, frame)
        temp_features_stress = calc_temp_features(temp_stress, frame)

        ecg_features_freq_base = ECG_freq_data(ecg_base, frame)
        ecg_features_freq_stress = ECG_freq_data(ecg_stress, frame)

        acc_features_base = acc_features(acc_base, frame)
        acc_features_stress = acc_features(acc_stress, frame)

        resp_features_base = calc_resp_features(resp_base, 100, frame)
        resp_features_stress = calc_resp_features(resp_stress, 100, frame)

        base_dict['EDA'] = eda_features_base
        base_dict['EMG'] = emg_features_base
        base_dict['TEMP'] = temp_features_base
        base_dict['ECG'] = pd.concat([ecg_features_time_base, ecg_features_freq_base], axis = 1)
        base_dict['ACC'] = acc_features_base
        base_dict['RESP'] = resp_features_base

        stress_dict['EDA'] = eda_features_stress
        stress_dict['EMG'] = emg_features_stress
        stress_dict['TEMP'] = temp_features_stress
        stress_dict['ECG'] = pd.concat([ecg_features_time_stress, ecg_features_freq_stress], axis = 1)
        stress_dict['ACC'] = acc_features_stress
        stress_dict['RESP'] = resp_features_stress


        patient_stress_df = pd.concat(stress_dict, axis = 1)
        patient_base_df = pd.concat(base_dict, axis = 1)



        stress_state = np.append(np.zeros(patient_base_df.shape[0]) , np.ones(patient_stress_df.shape[0]))
        stress_out = {}
        stress_df = pd.DataFrame(stress_state, columns = ['Out'])
        stress_out['Out'] = stress_df
        stress_df = pd.concat(stress_out, axis = 1)
        patient_df = pd.concat([patient_df, pd.concat([patient_base_df, patient_stress_df], ignore_index = True),stress_df], axis = 1)


        subject_list = np.asarray([item]*len(patient_df.index))
        subject_name = {}
        subject_df = pd.DataFrame(subject_list, columns = ['Subject'])
        subject_name['Subject'] = subject_df
        subject_name_df = pd.concat(subject_name, axis = 1)

        gender_list = np.asarray([C4_gender[index_in_list]]*len(patient_df.index))
        gender = {}
        gender_temp = pd.DataFrame(gender_list, columns = ['Gender'])
        gender['Gender'] = gender_temp
        gender_df = pd.concat(gender, axis = 1)


        patient_df = pd.concat([subject_name_df, gender_df, patient_df], axis = 1)

        all_data_df = pd.concat([all_data_df, patient_df], ignore_index = True)
        


    all_data_df.to_csv('C4-features-windows/C4_' + str(frame) + '_sec.csv')


    wesad_data_df = pd.read_csv('WESAD_data_1_min.csv', header = ([0,1]), index_col=0)

    all_data_df = pd.read_csv('C4-features-windows/C4_' + str(frame) + '_sec.csv', header = ([0,1]), index_col=0)



    list5 = list(all_data_df.columns.values)
    list5.remove(('Subject', 'Subject'))
    list5.remove(('Out', 'Out'))
    list5.remove(('Gender', 'Gender'))

    for item in (C4_subjects):
        for i in range(len(list5)):
            c4_base_feature_mean = np.mean(all_data_df[ (all_data_df['Subject']['Subject'] == str(item)) & (all_data_df['Out']['Out'] == 0)][str(list(list5)[i][0])][str(list(list5)[i][1])])
            c4_stress_feature_mean = np.mean(all_data_df[ (all_data_df['Subject']['Subject'] == str(item)) & (all_data_df['Out']['Out'] == 1)][str(list(list5)[i][0])][str(list(list5)[i][1])])
            wesad_base_feature_mean = np.mean(wesad_data_df[wesad_data_df['Out']['Out'] == 0][str(list(list5)[i][0])][str(list(list5)[i][1])])
            scaling_factor = 1+ ((wesad_base_feature_mean - c4_base_feature_mean)/c4_base_feature_mean)
            all_data_df.loc[(all_data_df['Subject']['Subject'] == str(item)), (str(list(list5)[i][0]), [str(list(list5)[i][1])])]*= scaling_factor

    all_data_df.to_csv('C4-features-windows/C4_' + str(frame) + '_sec.csv')
    




P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12
P1
P2
P5
P9
P10
P11
P12


In [9]:
list5 = list(all_data_df.columns.values)
print(list(all_data_df.columns.values))

[('ACC', 'Max freq_x'), ('ACC', 'Max freq_y'), ('ACC', 'Max freq_z'), ('ACC', 'integral 3D'), ('ACC', 'integral_x'), ('ACC', 'integral_y'), ('ACC', 'integral_z'), ('ACC', 'mean_3D'), ('ACC', 'mean_x'), ('ACC', 'mean_y'), ('ACC', 'mean_z'), ('ACC', 'std 3D'), ('ACC', 'std_x'), ('ACC', 'std_y'), ('ACC', 'std_z'), ('ECG', 'HRV_CVNN'), ('ECG', 'HRV_CVSD'), ('ECG', 'HRV_HF'), ('ECG', 'HRV_HFn'), ('ECG', 'HRV_HTI'), ('ECG', 'HRV_IQRNN'), ('ECG', 'HRV_LF'), ('ECG', 'HRV_LFHF'), ('ECG', 'HRV_LFn'), ('ECG', 'HRV_LnHF'), ('ECG', 'HRV_MCVNN'), ('ECG', 'HRV_MadNN'), ('ECG', 'HRV_MaxNN'), ('ECG', 'HRV_MeanNN'), ('ECG', 'HRV_MedianNN'), ('ECG', 'HRV_MinNN'), ('ECG', 'HRV_Prc20NN'), ('ECG', 'HRV_Prc80NN'), ('ECG', 'HRV_RMSSD'), ('ECG', 'HRV_SDNN'), ('ECG', 'HRV_SDSD'), ('ECG', 'HRV_TINN'), ('ECG', 'HRV_VHF'), ('ECG', 'HRV_pNN20'), ('ECG', 'HRV_pNN50'), ('EDA', 'Area'), ('EDA', 'Dynamic_Range'), ('EDA', 'EDA_mean'), ('EDA', 'EDA_std'), ('EDA', 'No.Peaks'), ('EDA', 'Orienting_mag_mean'), ('EDA', 'Orien

In [11]:
list5.remove(('Subject', 'Subject'))
list5.remove(('Out', 'Out'))
list5.remove(('Gender', 'Gender'))



ValueError: list.remove(x): x not in list

In [13]:
print(len(list5))

95


In [4]:
all_data_df = pd.read_csv('C4-features-windows/C4_' + str(60) + '_sec.csv', header = ([0,1]), index_col=0)

#all_data_df = pd.read_csv('C4_1_min.csv', header = ([0,1]), index_col=0)


all_data_df


Subject Gender       EDA                                            \
    Subject Gender  EDA_mean       EDA_std   Phasic_mean    Phasic_std   
0        P1      2  3.668418  2.904271e-02  4.456738e-06  5.340573e-03   
1        P1      2  3.866186  3.830719e-02 -6.758634e-05  5.635028e-03   
2        P1      2  3.865714  6.140576e-03  8.168307e-06  1.155297e-03   
3        P1      2  3.891270  1.784215e-02 -3.143086e-06  1.645394e-03   
4        P1      2  3.964327  1.149496e-02  1.047313e-05  2.282087e-03   
..      ...    ...       ...           ...           ...           ...   
155     P12      2  3.854306  3.012896e-02 -5.296919e-06  4.633546e-03   
156     P12      2  3.854543  8.679289e-10 -6.230196e-13  1.808925e-10   
157     P12      2  3.854555  2.028604e-03  8.190304e-10  3.272032e-04   
158     P12      2  3.854543  8.679289e-10 -6.230196e-13  1.808925e-10   
159     P12      2  3.847963  3.260159e-01 -1.088499e-05  1.660868e-02   

                                                                ...       ACC  \
       No.Peaks          Area Dynamic_Range Orienting_mag_mean  ...   mean_3D   
0     31.169855  36215.670944  2.252820e-02       3.645822e-03  ...  0.112381   
1     26.919420  23161.934690  4.001277e-02       2.127174e-03  ...  0.111039   
2     22.668986  31615.087441  5.556190e-03       7.644664e-04  ...  0.109159   
3     26.919420  40344.771941  6.181797e-03       1.308712e-03  ...  0.108702   
4     25.502609  35221.239307  1.208793e-02       2.051189e-03  ...  0.109871   
..          ...           ...           ...                ...  ...       ...   
155    8.146667  15200.721900  4.944014e-02       1.218726e-03  ...  0.131563   
156  148.269333  26144.699637  1.790710e-09       2.432823e-15  ...  0.129710   
157    4.073333  14850.953258  2.867776e-03       3.873189e-04  ...  0.129943   
158  148.269333  26144.699637  1.790710e-09       2.432823e-15  ...  0.129225   
159    8.146667  26696.267578  1.433039e-01       7.374767e-03  ...  0.127569   

                                        RESP                                 \
       std 3D   integral 3D Respiration Rate Inahaltion Mean Inahaltion Std   
0    0.048331  18188.409543        14.330949        2.195722       0.556115   
1    0.024097  17971.078352        16.474595        2.165412       0.501323   
2    0.021968  17666.767464        20.242794        1.577268       0.305206   
3    0.020940  17592.841786        15.846223        1.637127       0.399744   
4    0.016417  17782.012941        16.758634        1.975709       0.505320   
..        ...           ...              ...             ...            ...   
155  0.014008  16603.395119        16.703132        2.078735       0.533303   
156  0.008358  16349.238042        13.487001        2.113943       0.433283   
157  0.007151  16363.878409        14.782651        2.278697       0.405414   
158  0.007211  16287.109091        16.544055        1.847265       0.344535   
159  0.021924  16481.179503        16.035007        1.641967       0.198003   

                                                 Out  
    Exhalation Mean Exhalation Mean.1     RMSSD  Out  
0          2.004008          0.893387  3.912122  0.0  
1          1.534047          0.790235  3.446667  0.0  
2          1.391630          0.426629  2.737137  0.0  
3          2.094094          1.818445  3.883104  0.0  
4          1.630550          0.875469  3.452600  0.0  
..              ...               ...       ...  ...  
155        1.566869          0.677562  3.383877  1.0  
156        2.302492          1.088531  4.097441  1.0  
157        1.752060          0.715316  3.651947  1.0  
158        1.755944          0.971285  3.334846  1.0  
159        2.020724          0.965542  3.381168  1.0  

[160 rows x 98 columns]

In [8]:
wesad_data_df = pd.read_csv('WESAD_data_1_min.csv', header = ([0,1]), index_col=0)

for item in (C4_subjects):
    for i in range(2,len(list(all_data_df.columns.values[2:-1]))):
        c4_base_feature_mean = np.mean(all_data_df[ (all_data_df['Subject']['Subject'] == str(item)) & (all_data_df['Out']['Out'] == 0)][str(list(all_data_df.columns.values)[i][0])][str(list(all_data_df.columns.values)[i][1])])
        c4_stress_feature_mean = np.mean(all_data_df[ (all_data_df['Subject']['Subject'] == str(item)) & (all_data_df['Out']['Out'] == 1)][str(list(all_data_df.columns.values)[i][0])][str(list(all_data_df.columns.values)[i][1])])
        wesad_base_feature_mean = np.mean(wesad_data_df[wesad_data_df['Out']['Out'] == 0][str(list(all_data_df.columns.values)[i][0])][str(list(all_data_df.columns.values)[i][1])])
        scaling_factor = 1+ ((wesad_base_feature_mean - c4_base_feature_mean)/c4_base_feature_mean)
        all_data_df.loc[(all_data_df['Subject']['Subject'] == str(item)), (str(list(all_data_df.columns.values)[i][0]), [str(list(all_data_df.columns.values)[i][1])])]*= scaling_factor

all_data_df.to_csv('C4-features-windows/C4_' + str(frame) + '_sec.csv')



In [9]:
all_data_df

Subject Gender       EDA                                            \
    Subject Gender  EDA_mean       EDA_std   Phasic_mean    Phasic_std   
0        P1      2  3.668418  2.904271e-02  4.456738e-06  5.340573e-03   
1        P1      2  3.866186  3.830719e-02 -6.758634e-05  5.635028e-03   
2        P1      2  3.865714  6.140576e-03  8.168307e-06  1.155297e-03   
3        P1      2  3.891270  1.784215e-02 -3.143086e-06  1.645394e-03   
4        P1      2  3.964327  1.149496e-02  1.047313e-05  2.282087e-03   
..      ...    ...       ...           ...           ...           ...   
155     P12      2  3.854306  3.012896e-02 -5.296919e-06  4.633546e-03   
156     P12      2  3.854543  8.679289e-10 -6.230196e-13  1.808925e-10   
157     P12      2  3.854555  2.028604e-03  8.190304e-10  3.272032e-04   
158     P12      2  3.854543  8.679289e-10 -6.230196e-13  1.808925e-10   
159     P12      2  3.847963  3.260159e-01 -1.088499e-05  1.660868e-02   

                                                                \
       No.Peaks          Area Dynamic_Range Orienting_mag_mean   
0     31.169855  36215.670944  2.252820e-02       3.645822e-03   
1     26.919420  23161.934690  4.001277e-02       2.127174e-03   
2     22.668986  31615.087441  5.556190e-03       7.644664e-04   
3     26.919420  40344.771941  6.181797e-03       1.308712e-03   
4     25.502609  35221.239307  1.208793e-02       2.051189e-03   
..          ...           ...           ...                ...   
155    8.146667  15200.721900  4.944014e-02       1.218726e-03   
156  148.269333  26144.699637  1.790710e-09       2.432823e-15   
157    4.073333  14850.953258  2.867776e-03       3.873189e-04   
158  148.269333  26144.699637  1.790710e-09       2.432823e-15   
159    8.146667  26696.267578  1.433039e-01       7.374767e-03   

                                                                        \
    Orienting_mag_std orient_time_mean orient_time_std recov_time_mean   
0        3.977755e-03         1.277127    1.509410e+00        0.472027   
1        4.995553e-03         1.683646    3.134634e+00        0.550832   
2        8.268366e-04         1.723285    1.384866e+00        0.709865   
3        1.250809e-03         1.799396    2.047789e+00        0.780054   
4        2.018662e-03         1.749637    1.743567e+00        0.636682   
..                ...              ...             ...             ...   
155      1.668246e-03         4.550825    6.488068e+00        1.867209   
156      2.598135e-15         0.003981    9.542737e-19        0.001993   
157      6.816648e-04         3.716617    7.160156e+00        1.410319   
158      2.598135e-15         0.003981    9.542737e-19        0.001993   
159      1.540142e-02         4.649951    1.002405e+01        1.489025   

                           EMG                                          \
    recov_time_std         MNF         MDF PeakFrequency           MNP   
0     5.311721e-01  119.284251  121.979167     81.892152  2.533995e-06   
1     6.692821e-01  122.637338  103.682292     62.623410  1.309068e-06   
2     3.757484e-01  120.120728  103.682292     67.440596  1.606054e-06   
3     1.119208e+00  122.567639  103.682292     62.623410  1.378917e-06   
4     6.821514e-01  124.085541  103.682292     62.623410  1.234615e-06   
..             ...         ...         ...           ...           ...   
155   3.166666e+00  129.081292   97.274525     63.689159  3.836309e-08   
156   4.243990e-19  136.342548  101.327631     77.842306  3.232168e-08   
157   2.269868e+00  140.549529  105.380736     77.842306  3.086460e-08   
158   4.243990e-19  137.040709  105.380736     81.380592  3.250147e-08   
159   3.033489e+00   99.446856   97.274525     67.227446  3.278470e-07   

                                                                          \
          TTP       SM1           SM2           SM3         FR       PSR   
0    0.000327  0.036351  3.490308e-07  6.724834e-12  32.024190  0.339645   
1    0.000169  0.019307  8.4

In [105]:
c4_data_df.head(50)

Subject Gender       EDA                                                \
   Subject Gender  EDA_mean   EDA_std   Phasic_mean Phasic_std   No.Peaks   
0       P1      2  3.668418  0.029043  4.456738e-06   0.005341  31.169855   
1       P1      2  3.866186  0.038307 -6.758634e-05   0.005635  26.919420   
2       P1      2  3.865714  0.006141  8.168307e-06   0.001155  22.668986   
3       P1      2  3.891270  0.017842 -3.143086e-06   0.001645  26.919420   
4       P1      2  3.964327  0.011495  1.047313e-05   0.002282  25.502609   
5       P1      2  3.961408  0.021290  7.396645e-06   0.004085  29.753043   
6       P1      2  4.112225  0.025454  2.433054e-05   0.002138  24.085797   
7       P1      2  3.845884  0.039041 -5.207856e-06   0.003771  35.420290   
8       P1      2  3.636760  0.030193  7.772154e-06   0.004994  35.420290   
9       P1      2  3.553053  0.020188 -8.532871e-05   0.004906  34.003478   
10      P1      2  3.579351  0.012592  2.685298e-05   0.003255  26.919420   
11      P1      2  3.506085  0.023925 -5.238092e-06   0.004966  28.336232   
12      P1      2  3.325770  0.043734  1.350522e-05   0.003992  24.085797   
13      P1      2  3.559376  0.044145 -5.868700e-05   0.004929  28.336232   
14      P1      2  3.714986  0.011924 -5.594466e-06   0.001315  31.169855   
15      P1      2  3.715776  0.014306 -5.208291e-06   0.001951  18.418551   
16      P1      2  3.627559  0.037055  6.268995e-06   0.005015  31.169855   
17      P1      2  3.753876  0.038516 -8.973279e-06   0.002621  22.668986   
18      P1      2  4.040361  0.013704 -9.968927e-07   0.000424  32.586667   
19      P1      2  4.135249  0.006538  1.786861e-07   0.000171  17.001739   
20      P1      2  4.201605  0.005073  1.184045e-06   0.000230  14.168116   
21      P1      2  4.247793  0.004646  3.127139e-06   0.000361  15.584928   
22      P1      2  4.284404  0.003143  1.753882e-06   0.000135  21.252174   
23      P1      2  4.309861  0.001377  2.015046e-06   0.000077  12.751304   
24      P2     -2  3.968927  0.029847 -1.687390e-05   0.005656  17.710145   
25      P2     -2  3.755179  0.019425 -1.548669e-05   0.002843  14.758454   
26      P2     -2  3.762651  0.021169  7.600346e-07   0.003586  30.992754   
27      P2     -2  3.872860  0.015696 -1.901969e-06   0.004028  30.992754   
28      P2     -2  3.988150  0.017294 -2.630564e-08   0.000674  41.323671   
29      P2     -2  4.341617  0.068741  8.986507e-06   0.009722  25.089372   
30      P2     -2  4.314542  0.041267  7.991107e-06   0.005594  25.089372   
31      P2     -2  4.139144  0.051027  2.908092e-06   0.012572  35.420290   
32      P2     -2  4.031990  0.060103 -8.625330e-06   0.008499  19.185990   
33      P2     -2  3.774823  0.109321  5.293029e-06   0.011065  20.661836   
34      P2     -2  3.182642  0.051522  6.219094e-06   0.007528  16.234300   
35      P2     -2  3.050550  0.057936 -9.216053e-06   0.009072  20.661836   
36      P2     -2  3.146564  0.037905 -6.195565e-06   0.004345  23.613527   
37      P2     -2  3.230990  0.037796  2.526476e-05   0.009645  23.613527   
38      P2     -2  3.288679  0.064308 -1.343844e-06   0.017426  20.661836   
39      P2     -2  3.602994  0.075336  5.379727e-05   0.023304  16.234300   
40      P2     -2  3.641067  0.069912  9.454771e-06   0.012988  22.137681   
41      P2     -2  4.422429  0.153026  7.515150e-05   0.038299  17.710145   
42      P2     -2  4.346935  0.119625 -1.720946e-05   0.011105  13.282609   
43      P2     -2  4.235974  0.114477  4.327108e-06   0.016884  14.758454   
44      P2     -2  4.484623  0.116232 -3.916537e-05   0.020504  22.137681   
45      P2     -2  4.484661  0.169047 -1.927310e-05   0.020156  10.330918   
46      P2     -2  3.872150  0.244774 -9.586008e-05   0.045417  17.710145   
47      P5      2  5.761925  0.004437  1.027132e-04   0.000948  18.476976   
48      P5      2  4.405616  0.033842  1.498921e-05   0.003465  23.516151   
49      P5      2  3.240901  0.018440  5.157935e-06   0.003813  33.594502   